In [ ]:
import pickle
import os
import pandas as pd
import numpy as np
np.random.seed(42)
from sklearn.model_selection import train_test_split as splt
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import EarlyStopping
import seaborn as sns
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [ ]:
df = pd.read_excel("Data/Milano_Cleaned_wSCORE.xlsx")

In [ ]:
df.sort_values("date_of_sampling")
df = df.select_dtypes(exclude=['object', 'datetime64'])
df = df.drop(labels = ['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
df = df.fillna(-1).replace(-1, np.nan).replace("-1", np.nan).replace(-1.0, np.nan)
df = df.dropna(how='any', axis=1)

In [ ]:
d = {i: df.loc[df.subject_id == i, df.columns] for i in range(1, df.subject_id.iat[-1])}
d = {dx: d[dx] for dx in d if d[dx].shape[0] != 0}
X = np.array([np.array(values)[:-1,:-1] for key, values in d.items()])
#X = np.array([X[:,:,i]/X[:,:,i].max() for i in range(X.shape[2])])
#print(X.shape)
#X = np.swapaxes(X, 0, 1)
#X = np.swapaxes(X, 1, 2)
y = np.array([np.array(values)[-1,-1] for key, values in d.items()])/100

In [ ]:
print(X.shape)
model = Sequential()
model.add(LSTM(16, input_shape = X.shape[1:], return_sequences=True, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(8, return_sequences=False, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'mean_squared_error', optimizer='adam')
print(y, y.shape)
print(X, X.shape)

In [ ]:
X_tr, X_ts, y_tr, y_ts = splt(X, y, test_size=0.25, random_state=42)
es = EarlyStopping(patience = 20, restore_best_weights = True)
history = model.fit(X_tr, y_tr, validation_data=(X_ts, y_ts), epochs = 340, callbacks = [es])